In [ ]:
# 13장 네트워크와 프로세스 간 통신 다루기
# -------------------------------------
# 하나의 컴퓨터에서 다른 컴퓨터와 데이터를 주고받으려면 통신이 필요하다. 
# 이번 장에서는 비동기, 소켓, 시그널과 관련된 통신 프로그램을 작성할 때 사용하는 파이썬 모듈을 알아본다.

## 071 비동기 방식으로 프로그래밍하려면? ― asyncio


In [ ]:
# 071 비동기 방식으로 프로그래밍하려면? ― asyncio

# asyncio는 async/await 구문을 사용하여 동시성 코드를 작성할 수 있게 해주는 모듈로, asyncio를 사용하면 단일 스레드 작업을 병렬로 처리할 수 있다.

# 파이썬 3.7 버전 이상부터 사용할 수 있다.

# 문제
# 다음은 서로 다른 입력 값으로 sum() 함수를 2번 수행하여 결괏값을 출력하는 파이썬 프로그램이다.

# import time


# def sleep():
#     time.sleep(1)


# def sum(name, numbers):
#     start = time.time()
#     total = 0
#     for number in numbers:
#         sleep()
#         total += number
#         print(f'작업중={name}, number={number}, total={total}')
#     end = time.time()
#     print(f'작업명={name}, 걸린시간={end-start}')
#     return total


# def main():
#     start = time.time()

#     result1 = sum("A", [1, 2])
#     result2 = sum("B", [1, 2, 3])

#     end = time.time()
#     print(f'총합={result1+result2}, 총시간={end-start}')


# if __name__ == "__main__":
#     main()

# sum() 함수에서 입력 값을 하나씩 더할 때마다 sleep() 함수에 의해 1초씩 시간이 소요된다. 이 프로그램을 실행한 결과는 다음과 같다.

# 작업중=A, number=1, total=1
# 작업중=A, number=2, total=3
# 작업명=A, 걸린시간=2.000162124633789
# 작업중=B, number=1, total=1
# 작업중=B, number=2, total=3
# 작업중=B, number=3, total=6
# 작업명=B, 걸린시간=3.0002427101135254
# 총합=9, 총시간=5.0004048347473145

# A 작업 2초, B 작업 3초 등 모두 5초의 시간이 걸리고 sum() 함수를 2번 호출한 결과의 총합은 9임을 알 수 있다.

# sum() 함수를 비동기 방식으로 호출하면 실행 시간을 줄일 수 있을 것이다. 
# 이 프로그램을 파이썬 비동기 모듈인 asyncio를 사용하여 비동기 방식으로 바꾸려면 어떻게 해야 할까?

In [4]:
import time

def sleep():
    time.sleep(1)
    
def sum(name, numbers):
    start = time.time()
    total = 0
    for number in numbers:
        sleep()
        total += number
        print(f'작업중={name}, number={number}, total={total}')
    end = time.time()
    print(f'작업명={name}, 걸린시간={end - start}')
    return total

def main():
    start = time.time()
    result1 = sum('A', [1,2])
    result2 = sum('B', [1,2,3])
    end = time.time()
    print(f'총합={result1+result2}, 총시간={end-start}')

if __name__ == '__main__':
    main()

작업중=A, number=1, total=1
작업중=A, number=2, total=3
작업명=A, 걸린시간=2.0010077953338623
작업중=B, number=1, total=1
작업중=B, number=2, total=3
작업중=B, number=3, total=6
작업명=B, 걸린시간=3.00077486038208
총합=9, 총시간=5.001782655715942


In [ ]:
# 풀이
# 다음은 앞의 코드에 asyncio 모듈을 적용한 문제 풀이이다.

# [파일명: asyncio_sample.py]

# import asyncio
# import time


# async def sleep():
#     await asyncio.sleep(1)


# async def sum(name, numbers):
#     start = time.time()
#     total = 0
#     for number in numbers:
#         await sleep()
#         total += number
#         print(f'작업중={name}, number={number}, total={total}')
#     end = time.time()
#     print(f'작업명={name}, 걸린시간={end-start}')
#     return total


# async def main():
#     start = time.time()

#     task1 = asyncio.create_task(sum("A", [1, 2]))
#     task2 = asyncio.create_task(sum("B", [1, 2, 3]))

#     await task1
#     await task2

#     result1 = task1.result()
#     result2 = task2.result()

#     end = time.time()
#     print(f'총합={result1+result2}, 총시간={end-start}')


# if __name__ == "__main__":
#     asyncio.run(main())

# 함수를 비동기로 호출하려면 이렇게 def 앞에 async라는 키워드를 넣으면 된다. 
# 그러면 이제 이 함수는 비동기 함수가 된다. 이때 async를 적용한 비동기 함수를 코루틴이라 부른다.

# 또한, 코루틴 안에서 다른 코루틴을 호출할 때는 await sleep()과 같이 await를 함수명 앞에 붙여 호출해야 한다. 
# 코루틴 수행 중 await 코루틴을 만나면 await로 호출한 코루틴이 종료될 때까지 기다리지 않고 제어권을 메인 스레드나 다른 코루틴으로 넘긴다. 
# 이러한 방식을 넌블록킹(non-blocking)이라 한다. 그리고 호출한 코루틴이 종료되면 이벤트에 의해 다시 그 이후 작업이 수행된다.

# 여기서 하나 눈여겨봐야 할 점은 sleep() 함수에서 time.sleep(1) 대신 asyncio.sleep(1)를 사용한 부분이다. 
# 코루틴이 아닌 time.sleep(1)을 사용한다면 await가 적용되지 않아 실행 시간을 줄일 수 없다.

# main() 함수에서 사용한 asyncio.create_task()는 수행할 코루틴 작업(태스크)을 생성한다. 
# 여기서는 작업을 생성할 뿐이지 실제로 코루틴이 수행되는 것은 아니다. 
# 실제 코루틴 실행은 await 태스크가 담당한다. 그리고 실행 태스크의 결괏값은 태스크.result()로 얻을 수 있다.

# asyncio.create_task()는 코루틴을 동시에 실행하는 데 꼭 필요하다. 
# 다음처럼 태스크가 아닌 await로 코루틴을 실행한다면 코루틴이 동시에 실행되지 않고 하나씩 차례로 실행되어 이득이 없을 것이다.

# result1 = await sum("A", [1, 2])
# result2 = await sum("B", [1, 2, 3])

# asyncio.run(main())은 런 루프를 생성하여 main() 코루틴을 실행한다. 
# 코루틴을 실행하려면 런 루프가 반드시 필요하다. 코루틴이 모두 비동기적으로 실행되기 때문에 그 시작과 종료를 감지할 수 있는 이벤트 루프가 반드시 필요하기 때문이다.

# 이 코드를 실행한 결과는 다음과 같다.

# c:\projects\pylib>python asyncio_sample.py
# 작업중=A, number=1, total=1
# 작업중=B, number=1, total=1
# 작업중=A, number=2, total=3
# 작업명=A, 걸린시간=2.000617742538452
# 작업중=B, number=2, total=3
# 작업중=B, number=3, total=6
# 작업명=B, 걸린시간=3.000927209854126
# 총합=9, 총시간=3.000927209854126
# A 작업과 B 작업을 교대로 호출한다. (제어권이 await에 의해 계속 바뀐다는 것을 알 수 있다.) 그리고 시간도 5초 걸리던 것이 3초만 걸리게 되므로 A, B 작업이 완전히 비동기적으로 동작했다는 것을 알 수 있다.

# 참고
# asyncio - 비동기 I/O: https://docs.python.org/ko/3/library/asyncio.html
# 코루틴과 태스크: https://docs.python.org/ko/3/library/asyncio-task.html